In [2]:
import numpy as np, os, sys, matplotlib.pyplot as plt, seaborn as sns, pandas, orjson
from tqdm.notebook import tqdm
from mosestokenizer import MosesTokenizer
from transformers import AutoTokenizer

tqdm.pandas()

import logging
logging.disable(logging.INFO)

In [3]:
def get_dict_of_df(df_dict, read_directory='../log_calculations/unified_outputs/', fname_to_read='filtered_df.csv'):
    
    dfs = {}
    read_dirs = []
    for exp, models in df_dict.items():
        for model in models:
            file_name = os.path.join(read_directory, exp, model, fname_to_read)
            read_dirs.append(os.path.join(read_directory, exp, model))
            dfs[model] = pandas.read_csv(file_name)
    return dfs, read_dirs

In [4]:
models = {
    # 'every_5_5000' : ['allenai/scibert_scivocab_cased', 'bert-base-cased', 'roberta-base', 'xlm-roberta-base']
    'every_5_5000' : ['bert-base-multilingual-cased']
}

dfs, read_dirs = get_dict_of_df(models, fname_to_read='filtered_df_with_source.tsv')

In [5]:
languages = pandas.concat([df['langdetect'] for _, df in dfs.items()], axis=0).drop_duplicates()
languages

0           en
19          de
205         fr
210         es
283         nl
634         id
962         af
1421        ca
1490        pt
1751        no
1893        fi
2078        it
2147        ja
2278        ru
2558        pl
3742        sv
4638        ko
5463        cy
8310        cs
9156        tr
9173     zh-cn
13944       hr
16109       et
19205       vi
20706       ar
21550       sl
22606       fa
23845       bg
24493       hu
24923       uk
29969       ro
32908       da
37183    zh-tw
37940       sk
43649       tl
43873       lv
46035       el
49571       lt
49682       mk
52079       th
56614       so
66688       he
69072       hi
77584       sw
96989       sq
Name: langdetect, dtype: object

In [11]:
tokenizers = {}
for lang in languages:
    tokenizers[lang] = MosesTokenizer(lang)
    


In [12]:
for model, df in dfs.items():
    tokenizer = AutoTokenizer.from_pretrained(model)
    df['abstract_input'] = df['tokens'].progress_apply(lambda x: tokenizer.decode(eval(x)[1:-1]))
    df['num_moses_tokens'] = df[['langdetect', 'abstract_input']].progress_apply(lambda x: len(tokenizers[x.langdetect](x.abstract_input)), axis=1)

  0%|          | 0/98041 [00:00<?, ?it/s]

  0%|          | 0/98041 [00:00<?, ?it/s]

In [13]:
for (model, df), dir in zip(dfs.items(), read_dirs):
    print(dir)
    print(model)
    df.to_csv(os.path.join(dir, 'filtered_df_with_moses_tokens.tsv'))

../log_calculations/unified_outputs/every_5_5000/bert-base-multilingual-cased
bert-base-multilingual-cased
